# Notebook For Training Character Recognition

### Datasets :
#### 1. Digits (0-9) : MNIST (28 * 28)
#### 2. Symbols ( ‘(’ , ‘)’, ‘-’, ‘+’, ‘*’ ): Kaggle Handwritten Mathematical Symbols Dataset (45 * 45)

### Preprocessing of symbols to match MNIST digits :
Converted to Binary 

Padded to 20 * 20 (preserving the aspect ratio)

Padded to 28 * 28 using Centre of mass

![image.png](attachment:image.png)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install patool
import patoolib
patoolib.extract_archive("drive/MyDrive/data.rar", outdir="data")

patool: Extracting drive/MyDrive/data.rar ...
patool: running /usr/bin/unrar x -- /content/drive/MyDrive/data.rar
patool:     with cwd='data'
patool: ... drive/MyDrive/data.rar extracted to `data'.


'data'

In [4]:
import tensorflow as tf

In [5]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm  
import keras
import math
from scipy import ndimage

### Defining Image Directory

In [6]:
train_dir = "data/extracted_images"


### Data : Collecting images

In [7]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

     
train_datagen = ImageDataGenerator(rescale=1./255,
    data_format='channels_first',
    validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(28, 28),
    color_mode = "grayscale",
    batch_size=20,
    shuffle = True,
    classes = ['0','1','2','3','4','5','6','7','8','9','+','-','times','(',')'],
    class_mode="sparse",
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(28, 28),
    color_mode = "grayscale",
    batch_size=20,
    shuffle = True,
    classes = ['0','1','2','3','4','5','6','7','8','9','+','-','times','(',')'],
    class_mode="sparse",
    subset='validation') 

Found 148219 images belonging to 15 classes.
Found 37047 images belonging to 15 classes.


 # Model

In [8]:
import keras
keras.backend.set_image_data_format('channels_first')

## ------------------------------------------Deep Learning Model : DCCNN-------------------------------------------

![image.png](attachment:image.png)

In [9]:
from keras.layers import merge, Input, concatenate
from keras.models import Model
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import MaxPooling2D, Convolution2D
img_rows, img_cols = 28, 28

nb_filters_1 = 64
nb_filters_2 = 128
nb_filters_3 = 256
nb_conv = 3
nb_conv_mid = 4
nb_conv_init = 5

init = Input(shape=(1, 28, 28),)

fork11 = Convolution2D(nb_filters_1, nb_conv_init, nb_conv_init,  activation="relu", padding='same')(init)
fork12 = Convolution2D(nb_filters_1, nb_conv_init, nb_conv_init, activation="relu", padding='same')(init)
merge1 = concatenate([fork11, fork12], axis=1, name='merge1')
# concat_feat = concatenate([concat_feat, x], mode='concat', axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
maxpool1 = MaxPooling2D(strides=(2,2), padding='same')(merge1)

fork21 = Convolution2D(nb_filters_2, nb_conv_mid, nb_conv_mid, activation="relu", padding='same')(maxpool1)
fork22 = Convolution2D(nb_filters_2, nb_conv_mid, nb_conv_mid, activation="relu", padding='same')(maxpool1)
merge2 = concatenate([fork21, fork22, ], axis=1, name='merge2')
maxpool2 = MaxPooling2D(strides=(2,2), padding='same')(merge2)

fork31 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", padding='same')(maxpool2)
fork32 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", padding='same')(maxpool2)
fork33 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", padding='same')(maxpool2)
fork34 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", padding='same')(maxpool2)
fork35 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", padding='same')(maxpool2)
fork36 = Convolution2D(nb_filters_3, nb_conv, nb_conv, activation="relu", padding='same')(maxpool2)
merge3 = concatenate([fork31, fork32, fork33, fork34, fork35, fork36, ], axis=1, name='merge3')
maxpool3 = MaxPooling2D(strides=(2,2), padding='same')(merge3)

dropout = Dropout(0.5)(maxpool3)

flatten = Flatten()(dropout)
output = Dense(15, activation="softmax")(flatten)

model = Model(inputs=init, outputs=output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 28, 28)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 6, 6)     1664        ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 64, 6, 6)     1664        ['input_1[0][0]']                
                                                                                                  
 merge1 (Concatenate)           (None, 128, 6, 6)    0           ['conv2d[0][0]',                 
                                                                  'conv2d_1[0][0]']           

In [10]:
from keras import optimizers
ada = tf.keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95)
model.compile(optimizer=ada,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              validation_steps=100,
                              epochs=10)

Epoch 1/10
100/100 [==============================] - 34s 33ms/step - loss: 2.4000 - accuracy: 0.1795 - val_loss: 2.2606 - val_accuracy: 0.1945
Epoch 2/10
100/100 [==============================] - 3s 32ms/step - loss: 2.0344 - accuracy: 0.3200 - val_loss: 1.7617 - val_accuracy: 0.3975
Epoch 3/10
100/100 [==============================] - 3s 29ms/step - loss: 1.6954 - accuracy: 0.4405 - val_loss: 1.3765 - val_accuracy: 0.5410
Epoch 4/10
100/100 [==============================] - 3s 29ms/step - loss: 1.4011 - accuracy: 0.5260 - val_loss: 1.2828 - val_accuracy: 0.5890
Epoch 5/10
100/100 [==============================] - 3s 32ms/step - loss: 1.1822 - accuracy: 0.6050 - val_loss: 1.0822 - val_accuracy: 0.6920
Epoch 6/10
100/100 [==============================] - 3s 29ms/step - loss: 1.0249 - accuracy: 0.6625 - val_loss: 1.0817 - val_accuracy: 0.6345
Epoch 7/10
100/100 [==============================] - 3s 28ms/step - loss: 0.8704 - accuracy: 0.7175 - val_loss: 0.8148 - val_accuracy: 0.741

## Validation Accuracy [10th Epoch] : ~97%

In [11]:
model.save('contents/DCNN_10AD_sy.h5')